# SpekIt Technical Challenge

## Business Problem

Pull the following from this link https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

* Taxi trip records from 4 sources

* Associated data dictionaries

* Taxi zone lookups

Answer the following questions about the effects of the Corona Virus on taxi transportation in NYC:

* What has been the economic impact of the pandemic on NYC taxi revenue over time?

* What was the most expensive trip? Between what zones? Conversely, what was the cheapest trip? What is the difference between these before and during the peak impact of the pandemic?

* What's the most popular payment method? Did this change because of the pandemic?

* What’s the most expensive day of the week to travel on? Did this change because of the pandemic?

Submit your answers in the form of a GitHub repository, with additional instructions on how to reproduce them. Store the data into an AWS Postgres instance (free tier) or equivalent and provide access to us to review.

## Importing packages

In [1]:
# Importing packages
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import gzip
import shutil
import os
import sqlite3
import db_to_sqlite
from sqlite3 import Error
import csv
from pathlib import Path
import subprocess
import io
from icecream import ic
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
import requests
from bs4 import BeautifulSoup
from functions_all import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Data

TLC Trip Record Data
The yellow and green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers authorized under the Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP). The trip data was not created by the TLC, and TLC makes no representations as to the accuracy of these data.

The For-Hire Vehicle (“FHV”) trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID (shape file below). These records are generated from the FHV Trip Record submissions made by bases. Note: The TLC publishes base trip record data as submitted by the bases, and we cannot guarantee or confirm their accuracy or completeness. Therefore, this may not represent the total amount of trips dispatched by all TLC-licensed bases. The TLC performs routine reviews of the records and takes enforcement actions when necessary to ensure, to the extent possible, complete and accurate information.


**Data Dictionaries and MetaData**
<br><br>
* Trip Record User Guide
* Yellow Trips Data Dictionary
* Green Trips Data Dictionary
* FHV Trips Data Dictionary
High Volume FHV Trips Data Dictionary
Taxi Zone Maps and Lookup Tables
Taxi Zone Lookup Table (CSV)
Taxi Zone Shapefile (CSV)
Taxi Zone Map – Bronx (JPG)
Taxi Zone Map – Brooklyn (JPG)
Taxi Zone Map – Manhattan (JPG)
Taxi Zone Map – Queens (JPG)
Taxi Zone Map – Staten Island (JPG)
Errata
09/08/2017 - FHV trip record files from June 2017 updated as of 09/08/2017
08/30/2017 - FHV trip record files from July 2016 through June 2017 updated as of 08/16/2017
03/13/2017 - FHV trip record files from January 2016 through December 2016 updated as of 02/14/2017
09/22/2015 - TPEP and LPEP trip data CSVs from January through June 2015 have been updated to include a new field [improvement_surcharge] which lists the itemized portion of the fare covering the Taxicab Improvement Surcharge or Street Hail Livery Improvement Surcharge. This is a $0.30 surcharge on all trips to help fund accessibility in taxis and SHLs, which began on January 1, 2015. All TPEP and LPEP trip data files uploaded moving forward will also include this new field.

**Due to the fact that fhv files and fhvhv files do not contain data that can be used in the project (financial info), I decided to use only green and yellow trip data. The other reason is that fhv and fhvhv files have a very limited span of usage**<br>
Also the fhv and fhvhv files switched columns and data included.<br>
* After 2019 the columns in the fhv_tripdata files were<br><br>
> dispatching_base_num<br>
pickup_datetime<br>
dropoff_datetime<br>
PULocationID<br>
DOLocationID<br>
SR_Flag<br>
* In 2018: the columns changed the order and another filed was added (not mentioned in the dictionary)<br>
* In 2017: the order and name of columns were the same as after 2019
* Before 2017: the files did not contain Dropoff location of Dropoff date/time


In [81]:
# #This line is to close the cursor if after previous runs it remains open
# cur.close()
# conn.close()

In [25]:
os.getcwd()

'C:\\Users\\elena\\Documents\\SpekIt_Tech_challenge\\NYC_taxi_problem\\Spekit-tech-challenge'

In [82]:
# Creating an empty database and initiating a cursor
conn = sqlite3.connect('data/sqlite/nyc_taxi_small.db')
cur = conn.cursor()


q="SELECT name FROM sqlite_master WHERE type='table'"
df=table_query(q, cur=cur)
df

Nothing was found


### Checking and cleaning green tripdata files and loading them into the database

#### 2021 green trip files, with manual check

In [106]:
files_ref=create_filelist('data/2021/',n=0)
files_ref

['data/2021/green_tripdata_2021-01.csv',
 'data/2021/green_tripdata_2021-02.csv',
 'data/2021/green_tripdata_2021-03.csv',
 'data/2021/green_tripdata_2021-04.csv',
 'data/2021/green_tripdata_2021-05.csv',
 'data/2021/green_tripdata_2021-06.csv',
 'data/2021/green_tripdata_2021-07.csv',
 'data/2021/yellow_tripdata_2021-01.csv',
 'data/2021/yellow_tripdata_2021-02.csv',
 'data/2021/yellow_tripdata_2021-03.csv',
 'data/2021/yellow_tripdata_2021-04.csv',
 'data/2021/yellow_tripdata_2021-05.csv',
 'data/2021/yellow_tripdata_2021-06.csv',
 'data/2021/yellow_tripdata_2021-07.csv']

In [107]:
df_green_2021 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "green_tripdata"
    if substring in fullstring:
        df_name='df_green_2021_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_green_2021=pd.concat(list_frames)

for df in list_frames:
    df=pd.DataFrame()

C:\Users\elena\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [108]:
df_green_2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570466 entries, 0 to 83690
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               321351 non-null  float64
 1   lpep_pickup_datetime   570466 non-null  object 
 2   lpep_dropoff_datetime  570466 non-null  object 
 3   store_and_fwd_flag     321351 non-null  object 
 4   RatecodeID             321351 non-null  float64
 5   PULocationID           570466 non-null  int64  
 6   DOLocationID           570466 non-null  int64  
 7   passenger_count        321351 non-null  float64
 8   trip_distance          570466 non-null  float64
 9   fare_amount            570466 non-null  float64
 10  extra                  570466 non-null  float64
 11  mta_tax                570466 non-null  float64
 12  tip_amount             570466 non-null  float64
 13  tolls_amount           570466 non-null  float64
 14  ehail_fee              0 non-null    

In [109]:
df_green_2021.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00
1,2.0,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75
2,2.0,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00
3,2.0,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00
4,2.0,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,NaN,-0.3,-52.80,3.0,1.0,0.00


In [110]:
df = df_green_2021[pd.notnull(df_green_2021['VendorID'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321351 entries, 0 to 51172
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               321351 non-null  float64
 1   lpep_pickup_datetime   321351 non-null  object 
 2   lpep_dropoff_datetime  321351 non-null  object 
 3   store_and_fwd_flag     321351 non-null  object 
 4   RatecodeID             321351 non-null  float64
 5   PULocationID           321351 non-null  int64  
 6   DOLocationID           321351 non-null  int64  
 7   passenger_count        321351 non-null  float64
 8   trip_distance          321351 non-null  float64
 9   fare_amount            321351 non-null  float64
 10  extra                  321351 non-null  float64
 11  mta_tax                321351 non-null  float64
 12  tip_amount             321351 non-null  float64
 13  tolls_amount           321351 non-null  float64
 14  ehail_fee              0 non-null    

In [111]:
df.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

**Dropping the following fields that seem to be irrelevant to the problem stated. Financially we are only interested in the total amount charged. I am also dropping the dropoff timestamp because the duration of a trip is not of interest to this particular problem**<br>
>lpep_dropoff_datetime<br>
store_and_fwd_flag<br>
RatecodeID<br>
fare_amount<br>
extra<br>
mta_tax<br>
tip_amount<br>
tolls_amount<br>
ehail_fee<br>
improvement_surcharge<br>
congestion_surcharge<br>
trip_type<br>
VendorID<br>
trip_distance<br>
passenger_count


**Last three fields are dropped based on the previous analysis (see the first notebook)**


In [112]:
#list of columns to drop
list_drp=['lpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge','congestion_surcharge',
          'trip_type','RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321351 entries, 0 to 51172
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   lpep_pickup_datetime  321351 non-null  object 
 1   PULocationID          321351 non-null  int64  
 2   DOLocationID          321351 non-null  int64  
 3   total_amount          321351 non-null  float64
 4   payment_type          321351 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 14.7+ MB


In [113]:
df.head()

,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,2021-01-01 00:15:56,43,151,6.80,2.0
1,2021-01-01 00:25:59,166,239,16.86,1.0
2,2021-01-01 00:45:57,41,42,8.30,1.0
3,2020-12-31 23:57:51,168,75,9.30,2.0
4,2021-01-01 00:16:36,265,265,-52.80,3.0


In [114]:
df.payment_type.unique()

array([2., 1., 3., 4., 5.])

**This column should not be float type nut integer. Changing data types of several fields**<br>
>payment_type to integer

In [115]:
df=df.astype({'payment_type': 'int32'})

df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])


In [116]:
# Cleaning up the memory of the large dataframe
df_yellow_2021 = pd.DataFrame()

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321351 entries, 0 to 51172
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  321351 non-null  datetime64[ns]
 1   PULocationID          321351 non-null  int64         
 2   DOLocationID          321351 non-null  int64         
 3   total_amount          321351 non-null  float64       
 4   payment_type          321351 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 13.5 MB


In [118]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

13

In [119]:
df=df.drop_duplicates()

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321338 entries, 0 to 51172
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  321338 non-null  datetime64[ns]
 1   PULocationID          321338 non-null  int64         
 2   DOLocationID          321338 non-null  int64         
 3   total_amount          321338 non-null  float64       
 4   payment_type          321338 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 23.5 MB


In [129]:
# Randomly eliminating 25% of data

df=df.sample(frac = 0.75, random_state=123)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241004 entries, 26242 to 30487
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  241004 non-null  datetime64[ns]
 1   PULocationID          241004 non-null  int64         
 2   DOLocationID          241004 non-null  int64         
 3   total_amount          241004 non-null  float64       
 4   payment_type          241004 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 10.1 MB


In [131]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239282 entries, 26242 to 30487
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  239282 non-null  datetime64[ns]
 1   PULocationID          239282 non-null  int64         
 2   DOLocationID          239282 non-null  int64         
 3   total_amount          239282 non-null  float64       
 4   payment_type          239282 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 10.0 MB


In [132]:
df=df_filtered

In [133]:
#Loading the pre-processed dataframe into the database
sqlite_table = "green_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

q="SELECT name FROM sqlite_master WHERE type='table'"
df=table_query(q, cur=cur)
df

,name
0,green_tripdata


In [134]:
q="""SELECT count(*) FROM green_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,239282


In [135]:
q="""SELECT * FROM green_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1
5,33252,2021-01-26 16:21:38,65,189,9.80,2
6,24603,2021-06-14 18:01:37,74,237,17.05,2
7,39327,2021-06-22 19:19:31,119,159,17.80,1
8,34278,2021-06-19 18:15:47,49,97,6.80,2
9,39478,2021-06-22 20:52:25,130,205,14.80,1


#### 2020

In [145]:
#Creating dataframes from all of the 2020 monthly files and concateneting them together for further pre-processing


files_ref=create_filelist('data/2020/',n=0)


df_green_2020 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "green_tripdata"
    if substring in fullstring:
        df_name='df_green_2020_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_green_2020=pd.concat(list_frames)

for df in list_frames:
    df=pd.DataFrame()

In [146]:
df_green_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1734051 entries, 0 to 83129
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             float64
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        float64
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  ehail_fee              float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  payment_type           float64
 18  trip_type              float64
 19  congestion_surcharge   float64
dtypes: float64(15), int64(2), object(3)
memory usage: 277.8+ MB


In [147]:
#Cleaning the dataframe from records with NULL values in VendorID fiels, removing unneeded columns and changing datatypes

df = df_green_2020[pd.notnull(df_green_2020['VendorID'])]

#list of columns to drop
list_drp=['lpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge','congestion_surcharge',
          'trip_type','RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)
df.info()

df=df.astype({'payment_type': 'int32'})

df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205959 entries, 0 to 46291
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   lpep_pickup_datetime  1205959 non-null  object 
 1   PULocationID          1205959 non-null  int64  
 2   DOLocationID          1205959 non-null  int64  
 3   total_amount          1205959 non-null  float64
 4   payment_type          1205959 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 55.2+ MB


In [148]:
#Cleaning up the dataframe to free the memory
df_green_2020=pd.DataFrame()

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205959 entries, 0 to 46291
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   lpep_pickup_datetime  1205959 non-null  datetime64[ns]
 1   PULocationID          1205959 non-null  int64         
 2   DOLocationID          1205959 non-null  int64         
 3   total_amount          1205959 non-null  float64       
 4   payment_type          1205959 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 50.6 MB


In [150]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

22

In [151]:
df=df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205937 entries, 0 to 46291
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   lpep_pickup_datetime  1205937 non-null  datetime64[ns]
 1   PULocationID          1205937 non-null  int64         
 2   DOLocationID          1205937 non-null  int64         
 3   total_amount          1205937 non-null  float64       
 4   payment_type          1205937 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 50.6 MB


In [152]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 904453 entries, 115434 to 15129
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  904453 non-null  datetime64[ns]
 1   PULocationID          904453 non-null  int64         
 2   DOLocationID          904453 non-null  int64         
 3   total_amount          904453 non-null  float64       
 4   payment_type          904453 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 38.0 MB


In [153]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 904453 entries, 115434 to 15129
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  904453 non-null  datetime64[ns]
 1   PULocationID          904453 non-null  int64         
 2   DOLocationID          904453 non-null  int64         
 3   total_amount          904453 non-null  float64       
 4   payment_type          904453 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 38.0 MB


In [154]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898213 entries, 115434 to 15129
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  898213 non-null  datetime64[ns]
 1   PULocationID          898213 non-null  int64         
 2   DOLocationID          898213 non-null  int64         
 3   total_amount          898213 non-null  float64       
 4   payment_type          898213 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 37.7 MB


In [155]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898213 entries, 115434 to 15129
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   lpep_pickup_datetime  898213 non-null  datetime64[ns]
 1   PULocationID          898213 non-null  int64         
 2   DOLocationID          898213 non-null  int64         
 3   total_amount          898213 non-null  float64       
 4   payment_type          898213 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 37.7 MB


In [156]:
#Loading into the database
sqlite_table = "green_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

In [157]:
q="""SELECT count(*) FROM green_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,1137495


In [158]:
q="""SELECT * FROM green_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1
5,33252,2021-01-26 16:21:38,65,189,9.80,2
6,24603,2021-06-14 18:01:37,74,237,17.05,2
7,39327,2021-06-22 19:19:31,119,159,17.80,1
8,34278,2021-06-19 18:15:47,49,97,6.80,2
9,39478,2021-06-22 20:52:25,130,205,14.80,1


#### 2019

In [159]:
files_ref=create_filelist('data/2019/',n=0)

df_green_2019 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "green_tripdata"
    if substring in fullstring:
        df_name='df_green_2019_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_green_2019=pd.concat(list_frames)

for df in list_frames:
    df=pd.DataFrame()

C:\Users\elena\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [160]:
df_green_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6044050 entries, 0 to 450626
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             float64
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        float64
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  ehail_fee              float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  payment_type           float64
 18  trip_type              float64
 19  congestion_surcharge   float64
dtypes: float64(15), int64(2), object(3)
memory usage: 968.4+ MB


In [161]:
df = df_green_2019[pd.notnull(df_green_2019['VendorID'])]

#list of columns to drop
list_drp=['lpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge','congestion_surcharge',
          'trip_type','RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)

df=df.astype({'payment_type': 'int32'})

df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5629943 entries, 0 to 359901
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 236.2 MB


In [162]:
#Cleaning up to free the memory
df_green_2019=pd.DataFrame()

In [163]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

157

In [164]:
df=df.drop_duplicates()

In [165]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4222340 entries, 371578 to 396412
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 177.2 MB


In [166]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4222340 entries, 371578 to 396412
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 177.2 MB


In [167]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4198671 entries, 371578 to 396412
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 176.2 MB


In [168]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4198671 entries, 371578 to 396412
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 176.2 MB


In [169]:
sqlite_table = "green_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

In [170]:
q="""SELECT count(*) FROM green_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,5336166


In [171]:
q="""SELECT * FROM green_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1
5,33252,2021-01-26 16:21:38,65,189,9.80,2
6,24603,2021-06-14 18:01:37,74,237,17.05,2
7,39327,2021-06-22 19:19:31,119,159,17.80,1
8,34278,2021-06-19 18:15:47,49,97,6.80,2
9,39478,2021-06-22 20:52:25,130,205,14.80,1


#### 2018

In [172]:
files_ref=create_filelist('data/2018/',n=0)

df_green_2018 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "green_tripdata"
    if substring in fullstring:
        df_name='df_green_2018_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_green_2018=pd.concat(list_frames)

for df in list_frames:
    df=pd.DataFrame()

In [173]:
df = df_green_2018[pd.notnull(df_green_2018['VendorID'])]

#list of columns to drop
list_drp=['lpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
          'trip_type','RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)

df=df.astype({'payment_type': 'int32'})

df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807303 entries, 0 to 685372
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             int64  
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        int64  
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  ehail_fee              float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  payment_type           int64  
 18  trip_type              float64
dtypes: float64(10), int64(6), object(3)
memory usage: 1.3+ GB


In [176]:
#Cleaning up to free the memory
df_green_2018=pd.DataFrame()

In [177]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

222

In [179]:
df=df.drop_duplicates()

In [180]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6605311 entries, 446082 to 349183
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 277.2 MB


In [181]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6605311 entries, 446082 to 349183
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 277.2 MB


In [182]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6576837 entries, 446082 to 349183
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 276.0 MB


In [183]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6576837 entries, 446082 to 349183
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   lpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   total_amount          float64       
 4   payment_type          int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 276.0 MB


In [184]:
sqlite_table = "green_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')


In [185]:
q="""SELECT count(*) FROM green_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,11913003


In [186]:
q="""SELECT * FROM green_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1
5,33252,2021-01-26 16:21:38,65,189,9.80,2
6,24603,2021-06-14 18:01:37,74,237,17.05,2
7,39327,2021-06-22 19:19:31,119,159,17.80,1
8,34278,2021-06-19 18:15:47,49,97,6.80,2
9,39478,2021-06-22 20:52:25,130,205,14.80,1


#### Indexing the table green_tripdata

In [188]:
df_=pd.DataFrame()
df_filtered=pd.DataFrame()

In [187]:
sql = ("CREATE INDEX index_green_ven1 ON green_tripdata (PULocationID, lpep_pickup_datetime);")
cur.execute(sql)

sql = ("CREATE INDEX index_green_ven2 ON green_tripdata (PULocationID, DOLocationID);")
cur.execute(sql)

sql = ("CREATE INDEX index_green_ven3 ON green_tripdata (lpep_pickup_datetime);")
cur.execute(sql)

sql = ("CREATE INDEX index_green_ven4 ON green_tripdata (total_amount);")
cur.execute(sql)

In [189]:
q="""SELECT * FROM green_tripdata"""
df_=table_query(q, cur=cur)
df_

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1
...,...,...,...,...,...,...
11912998,536916,2018-02-20 13:43:37,65,100,26.30,1
11912999,101094,2018-09-05 23:21:41,83,56,10.80,1
11913000,625040,2018-09-29 07:18:29,129,82,5.80,2
11913001,10026,2018-05-01 13:35:27,33,39,32.80,2


In [ ]:
df_=pd.DataFrame()

### Loading the yellow_tripdata into the database

#### 2021, with manual check

In [190]:
files_ref=create_filelist('data/2021/',n=0)

df_yellow_2021 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "yellow_tripdata"
    if substring in fullstring:
        df_name='df_yellow_2021_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_yellow_2021=pd.concat(list_frames)

for df in list_frames:
    df=pd.DataFrame()

C:\Users\elena\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [191]:
df_yellow_2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000700 entries, 0 to 2821514
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 2.1+ GB


In [192]:
df_yellow_2021.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


In [75]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

**Dropping the following fields that seem to be irrelevant to the problem stated. Financially we are only interested in the total amount charged.**<br>
>tpep_dropoff_datetime<br>
store_and_fwd_flag<br>
RatecodeID<br>
fare_amount<br>
extra<br>
mta_tax<br>
tip_amount<br>
tolls_amount<br>
improvement_surcharge<br>
congestion_surcharge<br>

In [194]:
df = df_yellow_2021[pd.notnull(df_yellow_2021['VendorID'])]

#list of columns to drop
list_drp=['tpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'improvement_surcharge','congestion_surcharge',
          'RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)

df=df.astype({'payment_type': 'int32'})

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166672 entries, 0 to 2690858
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 594.5 MB


In [195]:
# Cleaning up the memory of the large dataframe
df_yellow_2021 = pd.DataFrame()

In [196]:
df.head()

,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,2021-01-01 00:30:10,142,43,2,11.80
1,2021-01-01 00:51:20,238,151,2,4.30
2,2021-01-01 00:43:30,132,165,1,51.95
3,2021-01-01 00:15:48,138,132,1,36.35
4,2021-01-01 00:31:49,68,33,1,24.36


In [197]:
df.payment_type.unique()

array([2, 1, 4, 3, 5])

In [198]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

205

In [199]:
df=df.drop_duplicates()

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166467 entries, 0 to 2690858
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 594.4 MB


In [201]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10624850 entries, 1960497 to 1572249
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 445.8 MB


In [202]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10624850 entries, 1960497 to 1572249
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 445.8 MB


In [203]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10570475 entries, 1960497 to 1572249
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 443.6 MB


In [204]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10570475 entries, 1960497 to 1572249
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 443.6 MB


In [205]:
#Loading the pre-processed dataframe into the database
sqlite_table = "yellow_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

q="SELECT name FROM sqlite_master WHERE type='table'"
df=table_query(q, cur=cur)
df

,name
0,green_tripdata
1,yellow_tripdata


In [206]:
q="""SELECT count(*) FROM yellow_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,10570475


In [207]:
q="""SELECT * FROM yellow_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,1960497,2021-04-30 00:23:36,79,225,1,26.80
1,577771,2021-06-07 17:57:49,230,142,1,10.56
2,1493844,2021-05-20 14:03:34,140,164,1,18.80
3,1338621,2021-05-18 16:56:36,75,41,2,5.30
4,1234566,2021-01-30 18:29:03,230,163,2,8.80
5,1683861,2021-04-26 11:30:08,137,80,2,23.80
6,1778177,2021-06-20 22:49:57,79,137,1,11.16
7,2033053,2021-05-26 21:10:06,100,163,2,15.80
8,996616,2021-06-12 02:56:02,230,239,1,11.88
9,809463,2021-06-10 09:51:57,132,75,1,75.70


#### 2020

In [208]:
#Creating dataframes from all of the 2020 monthly files and concateneting them together for further pre-processing

files_ref=create_filelist('data/2020/',n=0)


df_yellow_2020 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "yellow_tripdata"
    if substring in fullstring:
        df_name='df_yellow_2020_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_yellow_2020=pd.concat(list_frames)

#Cleaning up monthly DFs
for df in list_frames:
    df=pd.DataFrame()

df_yellow_2020.info()

C:\Users\elena\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24648499 entries, 0 to 1461896
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 3.5+ GB


In [209]:
df = df_yellow_2020[pd.notnull(df_yellow_2020['VendorID'])]

#list of columns to drop
list_drp=['tpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'improvement_surcharge','congestion_surcharge',
          'RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)

df=df.astype({'payment_type': 'int32'})

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23838931 entries, 0 to 1362440
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 1000.3 MB


In [210]:
# Cleaning up the memory of the large dataframe
df_yellow_2020 = pd.DataFrame()

In [211]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

689

In [212]:
df=df.drop_duplicates()

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23838242 entries, 0 to 1362440
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 1000.3 MB


In [214]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17878682 entries, 716472 to 1386423
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 750.2 MB


In [215]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17878682 entries, 716472 to 1386423
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 750.2 MB


In [216]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17803737 entries, 716472 to 1386423
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 747.1 MB


In [217]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17803737 entries, 716472 to 1386423
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 747.1 MB


In [218]:
#Loading the pre-processed dataframe into the database
sqlite_table = "yellow_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

In [219]:
q="""SELECT count(*) FROM yellow_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,28374212


In [220]:
q="""SELECT * FROM yellow_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,1960497,2021-04-30 00:23:36,79,225,1,26.80
1,577771,2021-06-07 17:57:49,230,142,1,10.56
2,1493844,2021-05-20 14:03:34,140,164,1,18.80
3,1338621,2021-05-18 16:56:36,75,41,2,5.30
4,1234566,2021-01-30 18:29:03,230,163,2,8.80
5,1683861,2021-04-26 11:30:08,137,80,2,23.80
6,1778177,2021-06-20 22:49:57,79,137,1,11.16
7,2033053,2021-05-26 21:10:06,100,163,2,15.80
8,996616,2021-06-12 02:56:02,230,239,1,11.88
9,809463,2021-06-10 09:51:57,132,75,1,75.70


#### 2019

In [221]:
#Creating dataframes from all of the 2019 monthly files and concateneting them together for further pre-processing

files_ref=create_filelist('data/2019/',n=0)


df_yellow_2019 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "yellow_tripdata"
    if substring in fullstring:
        df_name='df_yellow_2019_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_yellow_2019=pd.concat(list_frames)

#Cleaning up monthly DFs
for df in list_frames:
    df=pd.DataFrame()

df_yellow_2019.info()

C:\Users\elena\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 84399019 entries, 0 to 6896316
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 11.9+ GB


In [222]:
df = df_yellow_2019[pd.notnull(df_yellow_2019['VendorID'])]

#list of columns to drop
list_drp=['tpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'improvement_surcharge','congestion_surcharge',
          'RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)

df=df.astype({'payment_type': 'int32'})

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84152418 entries, 0 to 6845298
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 3.4 GB


In [223]:
# Cleaning up the memory of the large dataframe
df_yellow_2019 = pd.DataFrame()

In [224]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

3325

In [225]:
df=df.drop_duplicates()

In [226]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84149093 entries, 0 to 6845298
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 3.4 GB


In [227]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63111820 entries, 5636199 to 5274915
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 2.6 GB


In [228]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63111820 entries, 5636199 to 5274915
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 2.6 GB


In [229]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62969385 entries, 5636199 to 5274915
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 2.6 GB


In [230]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62969385 entries, 5636199 to 5274915
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 2.6 GB


In [231]:
#Loading the pre-processed dataframe into the database
sqlite_table = "yellow_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

In [232]:
q="""SELECT count(*) FROM yellow_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,91343597


In [233]:
q="""SELECT * FROM yellow_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,1960497,2021-04-30 00:23:36,79,225,1,26.80
1,577771,2021-06-07 17:57:49,230,142,1,10.56
2,1493844,2021-05-20 14:03:34,140,164,1,18.80
3,1338621,2021-05-18 16:56:36,75,41,2,5.30
4,1234566,2021-01-30 18:29:03,230,163,2,8.80
5,1683861,2021-04-26 11:30:08,137,80,2,23.80
6,1778177,2021-06-20 22:49:57,79,137,1,11.16
7,2033053,2021-05-26 21:10:06,100,163,2,15.80
8,996616,2021-06-12 02:56:02,230,239,1,11.88
9,809463,2021-06-10 09:51:57,132,75,1,75.70


#### 2018

In [234]:
#Creating dataframes from all of the 2019 monthly files and concateneting them together for further pre-processing

files_ref=create_filelist('data/2018/',n=0)


df_yellow_2018 = pd.DataFrame()
list_frames=[]
for i in range(0,len(files_ref)):
    fullstring = files_ref[i]
    substring = "yellow_tripdata"
    if substring in fullstring:
        df_name='df_yellow_2018_'+str(i+1)
        df_name=pd.read_csv(fullstring)
        list_frames.append(df_name)
df_yellow_2018=pd.concat(list_frames)

#Cleaning up monthly DFs
for df in list_frames:
    df=pd.DataFrame()

df_yellow_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102804250 entries, 0 to 8173230
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        int64  
 4   trip_distance          float64
 5   RatecodeID             int64  
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           int64  
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
dtypes: float64(8), int64(6), object(3)
memory usage: 13.8+ GB


In [235]:
df = df_yellow_2018[pd.notnull(df_yellow_2018['VendorID'])]

#list of columns to drop
list_drp=['tpep_dropoff_datetime','store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax',
          'tip_amount', 'tolls_amount', 'improvement_surcharge',
          'RatecodeID','VendorID','trip_distance','passenger_count']
df=df.drop(list_drp, axis=1)

df=df.astype({'payment_type': 'int32'})

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102804250 entries, 0 to 8173230
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 4.2 GB


In [236]:
# Cleaning up the memory of the large dataframe
df_yellow_2018 = pd.DataFrame()

In [237]:
#Checking for duplicates and eliminating them prior to the uploading to the database
df.duplicated().sum()

4814

In [238]:
df.drop_duplicates()

,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,2018-01-01 00:21:05,41,24,2,5.80
1,2018-01-01 00:44:55,239,140,2,15.30
2,2018-01-01 00:08:26,262,141,1,8.30
3,2018-01-01 00:20:22,140,257,2,34.80
4,2018-01-01 00:09:18,246,239,1,16.55
...,...,...,...,...,...
8173226,2018-12-31 23:40:45,161,237,2,8.80
8173227,2018-12-31 23:16:46,263,24,1,17.22
8173228,2018-12-31 23:29:01,41,43,2,6.30
8173229,2018-12-31 23:55:39,263,75,1,7.56


In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102804250 entries, 0 to 8173230
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 4.2 GB


In [240]:
# Randomly eliminating 25% of data

df_=df.sample(frac = 0.75, random_state=123)
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77103188 entries, 2861140 to 6945593
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 3.2 GB


In [241]:
df=df_
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77103188 entries, 2861140 to 6945593
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 3.2 GB


In [242]:
#Eliminating all records total_amount with equal or less 0
df_filtered = df[df['total_amount'] >0]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77041411 entries, 2861140 to 6945593
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 3.2 GB


In [243]:
df=df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77041411 entries, 2861140 to 6945593
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   PULocationID          int64         
 2   DOLocationID          int64         
 3   payment_type          int32         
 4   total_amount          float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 3.2 GB


In [244]:
#Loading the pre-processed dataframe into the database
sqlite_table = "yellow_tripdata"
df.to_sql(sqlite_table, conn, if_exists='append')

In [245]:
q="""SELECT count(*) FROM yellow_tripdata"""
df=table_query(q, cur=cur)
df

,count(*)
0,168385008


In [246]:
q="""SELECT * FROM yellow_tripdata limit 10"""
df=table_query(q, cur=cur)
df

,index,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,1960497,2021-04-30 00:23:36,79,225,1,26.80
1,577771,2021-06-07 17:57:49,230,142,1,10.56
2,1493844,2021-05-20 14:03:34,140,164,1,18.80
3,1338621,2021-05-18 16:56:36,75,41,2,5.30
4,1234566,2021-01-30 18:29:03,230,163,2,8.80
5,1683861,2021-04-26 11:30:08,137,80,2,23.80
6,1778177,2021-06-20 22:49:57,79,137,1,11.16
7,2033053,2021-05-26 21:10:06,100,163,2,15.80
8,996616,2021-06-12 02:56:02,230,239,1,11.88
9,809463,2021-06-10 09:51:57,132,75,1,75.70


#### Indexing the table yellow_tripdata

In [247]:
df_=pd.DataFrame()
df_filtered=pd.DataFrame()

In [248]:
sql = ("CREATE INDEX index_yellow_ven1 ON yellow_tripdata (PULocationID, tpep_pickup_datetime);")
cur.execute(sql)

sql = ("CREATE INDEX index_yellow_ven2 ON yellow_tripdata (PULocationID, DOLocationID);")
cur.execute(sql)

sql = ("CREATE INDEX index_yellow_ven3 ON yellow_tripdata (tpep_pickup_datetime);")
cur.execute(sql)

sql = ("CREATE INDEX index_yellow_ven4 ON yellow_tripdata (total_amount);")
cur.execute(sql)

In [249]:
conn.commit()

### Pickling DataFrames

In [250]:
q="""SELECT * FROM green_tripdata"""
df_green_tripdata=table_query(q, cur=cur)

In [251]:
df_green_tripdata.head()

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1


In [252]:
with open('data/pickled_dataframes/green_tripdata.pickle', 'wb') as f:
    pickle.dump(df_green_tripdata, f)

In [253]:
df_green_tripdata=pd.DataFrame()

In [255]:
with open('data/pickled_dataframes/green_tripdata.pickle', 'rb') as f:
    df_green_tripdata=pickle.load(f)
df_green_tripdata.head()

,index,lpep_pickup_datetime,PULocationID,DOLocationID,total_amount,payment_type
0,26242,2021-06-15 15:52:25,75,41,6.30,2
1,1065,2021-05-01 17:13:57,41,116,14.16,1
2,9893,2021-06-06 10:56:53,42,69,12.30,2
3,44168,2021-05-27 19:41:48,41,74,24.80,2
4,27532,2021-04-19 14:49:59,134,216,12.96,1


In [256]:
df_green_tripdata=pd.DataFrame()

In [257]:
q="""SELECT * FROM yellow_tripdata"""
df_yellow_tripdata=table_query(q, cur=cur)

In [258]:
df_yellow_tripdata.head()

,index,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,1960497,2021-04-30 00:23:36,79,225,1,26.80
1,577771,2021-06-07 17:57:49,230,142,1,10.56
2,1493844,2021-05-20 14:03:34,140,164,1,18.80
3,1338621,2021-05-18 16:56:36,75,41,2,5.30
4,1234566,2021-01-30 18:29:03,230,163,2,8.80


In [259]:
with open('data/pickled_dataframes/yellow_tripdata.pickle', 'wb') as f:
    pickle.dump(df_yellow_tripdata, f)

In [260]:
df_yellow_tripdata=pd.DataFrame()

In [261]:
with open('data/pickled_dataframes/yellow_tripdata.pickle', 'rb') as f:
    df_yellow_tripdata=pickle.load(f)
df_yellow_tripdata.head()

,index,tpep_pickup_datetime,PULocationID,DOLocationID,payment_type,total_amount
0,1960497,2021-04-30 00:23:36,79,225,1,26.80
1,577771,2021-06-07 17:57:49,230,142,1,10.56
2,1493844,2021-05-20 14:03:34,140,164,1,18.80
3,1338621,2021-05-18 16:56:36,75,41,2,5.30
4,1234566,2021-01-30 18:29:03,230,163,2,8.80


In [262]:
df_yellow_tripdata=pd.DataFrame()

### Conclusion

**Based on the results of this database creation :**<br>
* The database consists of 2 indexed tables: 
** green_tripdata with 11913003 records
** yellow_tripdata with 168385008 records
* They can be used for further analysis both visual and computational to answer the problem stated.
* The link to the EDA notebook is here

**Link to the Main Analysis notebook**<br>

[Additional data wrangling, visualization and answers to the questions asked](working_w_DFs.ipynb)